In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#Determine the number of players
player = len(purchase_data["SN"].unique())
player_num = pd.DataFrame({"Total Players": [player]})
player_num

,Total Players
0,576


In [3]:
#Find Purchasing Data

#Unique Items
items=len(purchase_data["Item ID"].unique())

#Average Price of Items Purchased
avg_price=purchase_data["Price"].mean()

#Number of Purchases
num_purchase=len(purchase_data["Price"])

#Total Revenue from Purchases
total_rev=purchase_data["Price"].sum()

#Create DataFrame
calc_df=pd.DataFrame({"Number of Unique Items":[items],
                      "Average Price": [avg_price],
                      "Number of Purchases": [num_purchase],
                      "Total Revenue": [total_rev]})
#Change format
calc_df = calc_df.style.format({"Average Price": "${:,.2f}",
                                "Total Revenue": "${:,.2f}"})
calc_df



,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [4]:
#Determine Gender Demographics


unique_player=pd.DataFrame({"SN":purchase_data["SN"], 
                            "Gender":purchase_data["Gender"],
                            "Age": purchase_data["Age"]})
unique_player=unique_player.drop_duplicates().reset_index(drop=True)

count=unique_player["Gender"].value_counts()

for i in range(len(count)):
    x=pd.DataFrame({"Total Count": [count[i]],
                    "Percentage of Players": [count[i]/player]},
                    index=[count.index[i]])
    if i==0:
        gender_dem=x
    else:
        gender_dem = gender_dem.append(x)

gender_dem = gender_dem.style.format({"Percentage of Players": "{:.2%}"})
gender_dem

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [5]:
#Purchasing Analysis by Gender

#Caluclate Count and Total cost of purchases
for i in range(len(unique_player["SN"])):
    total=0
    pcount=0
    for x in range(len(purchase_data["SN"])):
        if unique_player.SN[i]==purchase_data.SN[x]:
            total=total+purchase_data.Price[x]
            pcount=pcount+1
    if i == 0:
        count_df=pd.DataFrame({"Purchase Count": [pcount], 
                               "Purchase Total": [total]})
    else:
        temp_df=pd.DataFrame({"Purchase Count": [pcount], 
                              "Purchase Total": [total]},
                              index = [i])
        count_df=count_df.append(temp_df)
unique_player=unique_player.join(count_df)


In [6]:
#Separate counts and totals by gender
for i in range(len(count)):
    gender_count=0
    total_purchase=0
    gender_num=0
    for x in range(len(unique_player["SN"])):
        if count.index[i]==unique_player.Gender[x]:
            gender_count=gender_count + 1
            gender_num=gender_num + unique_player["Purchase Count"][x]
            total_purchase = total_purchase + unique_player["Purchase Total"][x]


    y=pd.DataFrame({"Purchase Count": [gender_num],
                    "Average Purchase Price": [total_purchase/gender_num],
                    "Total Purchase Value": [total_purchase],
                    "Avg Total Purchase per Person": [total_purchase/gender_count]},
                    index=[count.index[i]])
    if i==0:
        gender_purchase=y
    else:
        gender_purchase=gender_purchase.append(y)

#Output formatting
gender_purchase=gender_purchase.sort_index()
gender_purchase.index.name = "Gender"
gender_purchase=gender_purchase.style.format({"Average Purchase Price":"${:,.2f}",
                                              "Total Purchase Value": "${:,.2f}",
                                              "Avg Total Purchase per Person": "${:,.2f}"})
gender_purchase


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
#Age Demographics

#Labelling bins
bins_cuts=[0,9,14,19,24,29,34,39,1000]
bin_labels=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

unique_player['Age-Binned']=pd.cut(unique_player['Age'], bins=bins_cuts, labels=bin_labels)

bins=unique_player["Age-Binned"].value_counts()
bins=bins.sort_index()

#Calculate numbers per bin
for x in range(len(bins)):
    z=pd.DataFrame({"Total Count": [bins[x]], "Percentage of Players": [bins[x]/player]},
                    index=[bin_labels[x]])
    if x==0:
        Ages=z
    else:
        Ages=Ages.append(z)

Ages=Ages.style.format({"Percentage of Players": "{:.2%}"})



In [8]:
#Purchasing Analysis by Age

q=0
for i in bin_labels:
    pur_count=0
    tot_val=0
    person_count = 0
    for x in range(len(unique_player["SN"])):
        if i==unique_player["Age-Binned"][x]:
            pur_count=pur_count + unique_player["Purchase Count"][x]
            person_count = person_count + 1
            tot_val=tot_val + unique_player["Purchase Total"][x]
    
    a=pd.DataFrame({"Purchase Count": [pur_count],
                    "Average Purchase Price": [tot_val/pur_count],
                    "Total Purchase Value": [tot_val],
                    "Avg Total Purchase per Person": [tot_val/person_count]},index=[i])
    if q==0:
        Purchase_age=a
        q=1
    else:
        Purchase_age=Purchase_age.append(a)


Purchase_age.index.name="Age Ranges"
Purchase_age=Purchase_age.style.format({"Average Purchase Price":"${:,.2f}",
                                        "Total Purchase Value": "${:,.2f}",
                                        "Avg Total Purchase per Person": "${:,.2f}"})
Purchase_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [52]:
#Top Spenders List

#Calculate values
unique_player["Average Purchase Price"]=unique_player["Purchase Total"]/unique_player["Purchase Count"]
top_spender=pd.DataFrame({"Purchase Count": unique_player["Purchase Count"],
                          "Average Purchase Price": unique_player["Average Purchase Price"],
                          "Total Purchase Value": unique_player["Purchase Total"],
                          "SN": unique_player["SN"]})
top_spender=top_spender.set_index("SN")
top_spender=top_spender.sort_values("Total Purchase Value", ascending=False)
top_spender=top_spender.iloc[0:5,0:4]
top_spender=top_spender.style.format({"Average Purchase Price":"${:,.2f}",
                                        "Total Purchase Value": "${:,.2f}"})
top_spender


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [51]:
#Most popular Items

item=pd.DataFrame({"Item Name": purchase_data["Item Name"],
                          "Item ID": purchase_data["Item ID"]})
item=item.drop_duplicates().reset_index(drop=True)

del unique_item_display
del unique_item
q=0
for i in range(len(item["Item Name"])):
    itotal=0
    icount=0
    for x in range(len(purchase_data["Item Name"])):

        if item["Item Name"][i]==purchase_data["Item Name"][x]:
            itotal=itotal+purchase_data["Price"][x]
            icount=icount+1
    z=pd.DataFrame({"Purchase Count": [icount],
                    "Item Price": [itotal/icount],
                    "Total Purchase Value": [itotal]},
                    index=[i])
    if q==0:
        q=1
        item_prices=z
    else:
        item_prices=item_prices.append(z)
unique_item=item.join(item_prices)
unique_item=unique_item.set_index(["Item ID","Item Name"])
unique_item=unique_item.sort_values(["Purchase Count","Item ID"], ascending=False)
unique_item_display=unique_item.iloc[0:5,0:4]
unique_item_display=unique_item_display.style.format({"Item Price":"${:,.2f}",
                                              "Total Purchase Value": "${:,.2f}"})

unique_item_display


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [53]:
#Most Profitable Items

#Re-order by purchase value
unique_item=unique_item.sort_values(["Total Purchase Value"], ascending=False)
#Truncate for top 5
unique_item_profit=unique_item.iloc[0:5,0:4]
#Change format
unique_item_profit=unique_item_profit.style.format({"Item Price":"${:,.2f}",
                                              "Total Purchase Value": "${:,.2f}"})
unique_item_profit

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
